In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

의사결정 트리(Decision Tree)는 테스트를 위쪽에 놓고 해당 테스트로 발생 가능한 결과를 아래쪽으로 놓는다. 의사결정 트리는 또 다른 테스트가 될 수 있다. 즉, 상위 테스트로 판별하지 못한 데이터는 또 다른 테스트인 자식 테스트를 사용해서 분류가 가능하다.

의사결정 트리는 테스트 성능을 평가하기 위해 엔트로피(Entropy)라는 개념을 사용한다.  
엔트로피는 불순도 정도를 측정하며, 낮을수록 좋다. 이때 불순도는 노스에 서로 다른 데이터가 얼마나 섞여있는지 의미한다. 불순도가 낮을수록 데이터가 섞여있지 않는다는 것을 의미한다.

지니 계수(Gini Index)는 엔트로피와 함께 불순도를 측정하는 또 다른 방법이다.  
데이터 셋에서 랜덤으로 삭제한 데이터에 임의로 라벨을 정했을 때 틀릴 확률의 의미한다. 만일 해당 데이터 셋의 데이터가 모두 동일하게 라벨링되어 있다면 지니 계수는 0을 가지게 되며 이는 불순도가 0임을 의미한다.

와인 데이터를 사용해 와인 종류를 분류하는 모델을 생성하고 학습시킨다.

In [4]:
# 데이터 불러오기
raw_data = datasets.load_wine() # 사이킷런이 제공하는 와인 데이터를 불러온다.

# 피쳐, 레이블 데이터 저장
xData = raw_data.data # 피쳐 데이터를 저장한다.
yData = raw_data.target # 피쳐 데이터에 따른 레이블을 저장한다.
print(xData.shape, yData.shape)

# 학습 데이터와 테스트 데이터로 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# 데이터 표준화
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
x_train = std_scale.fit_transform(x_train) # 학습 데이터를 스케일러로 표준화 하고 적용한다.
x_test = std_scale.transform(x_test) # 테스트 데이터를 학습 데이터로 표준화 스케일러에 적용한다.

# 모델 생성 후 데이터 학습
from sklearn.tree import DecisionTreeClassifier # 의사결정 알고리즘을 사용하기 위해 import 한다.
# 의사결정 트리 모델을 만든다.
# 의사결정 트리 알고리즘 구동시 random 하게 특성들과 데이터 건수를 선택하는 로직이 포함되어 있기 때문에
# random_state 속성에 특정 값을 지정하여 수행시 마다 동일한 트리를 만들수 있도록 한다.
# 의사결정 트리는 과적합되기 쉬운 모델이므로 과적합 방지를 위해 max_depth 속성을 지정해서 트리의 최대 깊이를
# 지정해 주는 것이 좋다.
clf = DecisionTreeClassifier(random_state=0, max_depth=4)
# 표준화된 학습 데이터와 학습 데이터에 따른 레이블 데이터로 가우시안 나이브 베이즈 모델을 학습시킨다.
clf.fit(x_train, y_train)

(178, 13) (178,)
(133, 13) (45, 13) (133,) (45,)


DecisionTreeClassifier(max_depth=4, random_state=0)

학습된 모델로 테스트 데이터를 예측한다.

In [5]:
# predict() 메소드의 인수로 표준화된 테스트 데이터를 넘겨서 예측한다.
predict = clf.predict(x_test)
print(predict)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 1 0 1 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 1 0 1 1 1]


학습된 모델을 평가한다.

In [6]:
# 혼동 행렬
# confusion_matrix() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 혼동 행렬를 만든다.
conf_matrix = confusion_matrix(y_test, predict)
print(conf_matrix)

[[14  2  0]
 [ 0 20  1]
 [ 0  0  8]]


In [8]:
# 분류 리포트
# classification_report() 메소드의 인수를 테스트 데이터의 실제값, 예측값 순서로 넘겨서 분류 리포트를 만든다.
# target_names 속성으로 분류 리포트에 레이블의 실제값을 출력할 수 있다.
# class_report = classification_report(y_test, predict, target_names=raw_data.target_names) 
class_report = classification_report(y_test, predict, target_names=['A', 'B', 'C']) 
print(class_report)

              precision    recall  f1-score   support

           A       1.00      0.88      0.93        16
           B       0.91      0.95      0.93        21
           C       0.89      1.00      0.94         8

    accuracy                           0.93        45
   macro avg       0.93      0.94      0.93        45
weighted avg       0.94      0.93      0.93        45

